In [ ]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb




In [20]:
# Criando função para calcular as métricas
def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }

In [21]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    elif scale == 'YcrCb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    else:
        raise ValueError("Escala de cor não suportada.")

In [22]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor
def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário

                # Converta para a escala de cor desejada
                img = convert_color_scale(img, color_scale)

                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal

                images.append(img)
    return np.array(images)

In [23]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"
cancer_dir = os.path.join(data_dir, 'Serous')
img_extensions = ['.jpg', '.jpeg', '.png']


In [24]:
pastas= {
    'Clear_Cell': r"C:\Users\andre\Pictures\OvarianCancer\Clear_Cell",
    'Endometri': r"C:\Users\andre\Pictures\OvarianCancer\Endometri",
    'Mucinous': r"C:\Users\andre\Pictures\OvarianCancer\Mucinous",
    'Non_Cancerous': r"C:\Users\andre\Pictures\OvarianCancer\Non_Cancerous"
}


tds_pastas = ['Clear_Cell', 'Endometri', 'Mucinous', 'Non_Cancerous']
df_all_metrics = pd.DataFrame()
for outro in tds_pastas:

    # Carregar o modelo InceptionV3 pré-treinado
    base_model = VGG16(weights='imagenet', include_top=False)     # <----------------------------------------------------------

    # Extrair características usando o modelo VGG16
    def extract_features(model, X):
        features = model.predict(X)
        return features.reshape((features.shape[0], -1))


    #--------------------------------------------------------------------------------------------------------------------------------------

    # Carregamento de imagens e conversão para XYZ
    normal_images = load_images(pastas[outro], 'grayscale',img_extensions)        # <----------------------------------------------------------
    cancer_images = load_images(cancer_dir, 'grayscale',img_extensions)        # hsv<----------------------------------------------------------


    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_images = extract_features(base_model, all_images)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=40)

    X_train = X_train / 255.0
    X_test = X_test / 255.0
    #--------------------------------------------------------------------------------------------------------------------------------------



    # Para o SVM
    svm_model = SVC(random_state=41, kernel='rbf', C=1, gamma='scale')
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    metrics_svm = get_metrics(y_test, y_pred_svm)
    print(f"Métricas para serous vs {outro}: (Com SVM,vgg16, e svm)")
    print(metrics_svm)



    # Criar DataFrames e salvar os resultados em um xlsx 
    df_metrics_svm = pd.DataFrame([metrics_svm.values()], columns=metrics_svm.keys())
    df_metrics_svm.insert(0, 'Seroso VS ', outro)




    # Concatenar todos os DataFrames
    df_all_metrics = pd.concat([df_all_metrics,df_metrics_svm], ignore_index=True)

# Salvar o DataFrame em um arquivo Excel
df_all_metrics.to_excel('EmSerousOutros.xlsx', index=False)  # <----------------------------------------------------------

df_all_metrics



Métricas para serous vs Clear_Cell: (Com SVM,vgg16, e svm)
{'accuracy': 0.85, 'specificity': 1.0, 'recall': 0.7142857142857143, 'precision': 1.0, 'f1-score': 0.8333333333333333, 'kappa': 0.7037037037037037, 'auc-roc': 0.8571428571428572}
Métricas para serous vs Endometri: (Com SVM,vgg16, e svm)
{'accuracy': 0.925, 'specificity': 0.8421052631578947, 'recall': 1.0, 'precision': 0.875, 'f1-score': 0.9333333333333333, 'kappa': 0.8484848484848485, 'auc-roc': 0.9210526315789473}
Métricas para serous vs Mucinous: (Com SVM,vgg16, e svm)
{'accuracy': 0.825, 'specificity': 0.8947368421052632, 'recall': 0.7619047619047619, 'precision': 0.8888888888888888, 'f1-score': 0.8205128205128205, 'kappa': 0.6517412935323383, 'auc-roc': 0.8283208020050126}
Métricas para serous vs Non_Cancerous: (Com SVM,vgg16, e svm)
{'accuracy': 0.9, 'specificity': 0.8947368421052632, 'recall': 0.9047619047619048, 'precision': 0.9047619047619048, 'f1-score': 0.9047619047619048, 'kappa': 0.7994987468671679, 'auc-roc': 0.899

,Seroso VS,accuracy,specificity,recall,precision,f1-score,kappa,auc-roc
0,Clear_Cell,0.850,1.000000,0.714286,1.000000,0.833333,0.703704,0.857143
1,Endometri,0.925,0.842105,1.000000,0.875000,0.933333,0.848485,0.921053
2,Mucinous,0.825,0.894737,0.761905,0.888889,0.820513,0.651741,0.828321
3,Non_Cancerous,0.900,0.894737,0.904762,0.904762,0.904762,0.799499,0.899749
